In [1]:
import numpy as np
from matplotlib import pyplot as plt
from scipy.integrate import solve_ivp, solve_bvp
import matplotlib.animation as animation
from matplotlib.animation import FuncAnimation
from IPython.display import HTML
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
import matplotlib.patches as mpatches
%config InlineBackend.figure_format = 'retina'
import matplotlib
matplotlib.rcParams['animation.embed_limit'] = 40

In [2]:
def generate_fishing_grounds(n_grounds = 10, x_max = 100, y_max = 100):
    return np.random.rand(n_grounds, 2) * np.array([x_max, y_max])
def generate_ships(n_ships = 10, x_max = 100, y_max = 100):
    return np.random.rand(n_ships, 2) * np.array([x_max, y_max])
def generate_ships_at_harbor(h, n_ships = 10):
    """
    Returns an n_ships x 2 array of ships at the harbor h with gaussian noise
    """
    # return np.array([h + np.random.normal(0, 1, 2) for _ in range(n_ships)])
    return np.array([h for _ in range(n_ships)])
def calculate_ship_distance_from_grounds(ships, grounds):
    return np.linalg.norm(ships[:, np.newaxis] - grounds, axis=2)
def d_matrix(ships, grounds):
    # Vectorized difference calculation
    differences = grounds - ships[:, np.newaxis]
    distances = np.linalg.norm(differences, axis=2, keepdims=True)
    return differences / (distances + 1e-6)  # Adding small value to avoid division by zero
def ship_ground_gaussians(ships, grounds, alpha, b=1, c=0):
    distances = np.linalg.norm(ships[:, np.newaxis] - grounds, axis=2)
    return b*np.exp(-alpha * distances**2) + c
def calculate_ship_distance_from_ships(ships):
    return np.linalg.norm(ships[:, np.newaxis] - ships, axis=2)
def harbor_attraction(U, ships, h):
    diff = h - ships
    diff_norm = np.linalg.norm(diff, axis=1, keepdims=True)
    return U[:, -2, np.newaxis] * (diff / (diff_norm + 1e-6))
def attraction_matrix(U, G, D, F):
    F_reshaped = F.reshape(1, -1, 1)
    return U[:, :-2, None] * G[:, :, None] * D * F_reshaped
def calculate_F_prime(t, F, C, rate):
    C_sum = np.sum(C, axis=0)
    return -rate * F * C_sum
def calculate_S_matrix(A, H):
    return np.sum(A, axis=1).reshape(-1, 2) + H
def calculate_x_prime(U, S):

    # Reshape U[:,-2] to have an extra dimension
    U_reshaped = U[:,-1].reshape(-1, 1)

    return U_reshaped * S
# LIKELY WRONG

def Dxs_matrix(x, grounds, ships, G, U, F, distances, differences, alpha, h):
    F_reshaped = F.reshape(1, -1, 1)
    
    # This will have shape (N, K, 1)
    outer_scalar = (U[:, :-2, None] * G[:, :, None] * F_reshaped) / (distances + 1e-6)
    
    # Calculate the inner scalar term, reshaping it to (N, K, 1, 1) to broadcast with the outer product
    inner_scalar = (alpha + 1 / (distances[..., 0]**2 + 1e-6)).reshape(*distances.shape[:2], 1, 1)
    
    # Compute the outer products
    outer_products = differences[..., np.newaxis] * differences[..., np.newaxis, :]
    
    # Identity matrix needs to be (1, 1, 2, 2) to broadcast correctly
    I_2 = np.eye(2).reshape((1, 1, 2, 2))

    # print((inner_scalar * outer_products - I_2).shape)
    
    # The term_inside_summation should have shape (N, K, 2, 2)
    term_inside_summation = outer_scalar[..., np.newaxis] * (inner_scalar * outer_products - I_2)
    
    # Sum over the second axis, corresponding to summing over 'j' (the grounds)
    # This will result in shape (N, 2, 2)
    DxS = np.sum(term_inside_summation, axis=1)

    diff = h - ships
    diff_norm = np.linalg.norm(diff, axis=1, keepdims=True) + 1e-6
    
    u_ratio = U[:, -2] / diff_norm.squeeze()

    h_outer_products = diff[..., np.newaxis] * diff[..., np.newaxis, :]

    h_inner = np.sum(diff**2, axis=1)

    h_outer_products /= h_inner[:, np.newaxis, np.newaxis] + 1e-6


    I_2 = np.eye(2).reshape((1, 2, 2))
    
    return DxS + ((h_outer_products - I_2) * u_ratio[:, np.newaxis, np.newaxis])

def calculate_p_f_prime(t, pf, C, rate):
    C_sum = np.sum(C, axis=0)
    return rate * pf * C_sum
def calculate_p_prime(p, pf, S, U, Dxs, ship_differences, differences, distances, rate, sigma, beta):
    

    # Shape: (N, 2, 2), the outer product of S
    S_outer = S[:, :, np.newaxis] * S[:, np.newaxis, :]

    # Shape: (N, K, 2), the inner product of S with the differences
    S_inner = (np.linalg.norm(S, axis=1) + 1e-6)**2

    # Shape: (N, 2, 2)
    S_outer /= (S_inner[:, np.newaxis, np.newaxis] + 1e-6)
    # print(S_outer.shape)

    # Shape: (N, 2, 2)
    U_scalar = U[:, -1][:, np.newaxis, np.newaxis]

    # Shape: (N, 2, 2)
    I_2 = np.eye(2).reshape((1, 2, 2))

    # Shape: (N, 2, 2)
    first_term = U_scalar * (I_2 - S_outer)

    # Shape: (N, 2, 2)
    dxs = (Dxs / (np.linalg.norm(S, axis=1)[:, np.newaxis, np.newaxis] + 1e-6))

    # Shape: (N, 2, 1)
    second_term = dxs @ p[:, :, np.newaxis]
# 
    # print(first_term.shape, second_term.shape)

    # Shape: (N, 2, 1)
    combined = np.matmul(first_term, second_term)

    # Shape: (N, N, 2)
    term_inside_summation = ship_differences / (np.linalg.norm(ship_differences, axis=1)**2 + 1e-6)**2

    # Shape: (N, 2)
    distance_sum = np.sum(term_inside_summation, axis=1)

    # Shape: (N, K, 2)
    inner_sum = pf[:,np.newaxis] * np.exp(-sigma * distances**2) * differences

    # Shape: (N, 2)
    sum_term = np.sum(inner_sum, axis=1) * 2*sigma*rate

    return (4 * beta * distance_sum) - sum_term - combined.squeeze()

    
def initialize_all(y0, t0, t, n):
    """ An initialization routine for the different ODE solving
    methods in the lab. This initializes Y, T, and h. """
    if isinstance(y0, np.ndarray):
        Y = np.empty((n, y0.size)).squeeze()
    else:
        Y = np.empty(n)
    
    Y[0] = y0
    T = np.linspace(t0, t, n)
    h = float(t - t0) / (n - 1)
    return Y, T, h

def RK4(f, y0, t0, t, n):
    """ Use the RK4 method to compute an approximate solution
    to the ODE y' = f(t, y) at n equispaced parameter values from t0 to t
    with initial conditions y(t0) = y0.
    
    y0 is assumed to be either a constant or a one-dimensional numpy array.
    t and t0 are assumed to be constants.
    f is assumed to accept three arguments.
    The first is a constant giving the value of t.
    The second is a one-dimensional numpy array of the same size as y.
    The third is an index to the other arrays.
    
    This function returns an array Y of shape (n,) if
    y is a constant or an array of size 1.
    It returns an array of shape (n, y.size) otherwise.
    In either case, Y[i] is the approximate value of y at
    the i'th value of np.linspace(t0, t, n).
    """
    Y,T,h = initialize_all(y0,t0,t,n)
    for i in range(n-1):
        K1 = f(T[i],Y[i],i)
        K2 = f(T[i]+h/2.,Y[i]+h/2.*K1,i)
        K3 = f(T[i]+h/2.,Y[i]+h/2.*K2,i)
        K4 = f(T[i+1],Y[i]+h*K3,i)
        Y[i+1] = Y[i] + h/6.*(K1+2*K2 +2*K3+K4)
    return Y
def ode(t, y, params):
    alpha, sigma, beta, rate, U, grounds, h, x_size, F_size, p_size, x_shape, p_shape = params
    b = 1
    x = y[:x_size].reshape(x_shape)  # reshape x back to its original shape
    F = y[x_size:x_size+F_size]
    p = y[x_size + F_size: x_size + F_size + p_size].reshape(p_shape)
    pf = y[x_size + F_size + p_size:]

    differences = grounds - x[:, np.newaxis]
    distances = np.linalg.norm(differences, axis=2, keepdims=True)

    ship_diffences = x[:, np.newaxis] - x
    # ship_distances = np.linalg.norm(ship_diffences, axis=2)

    # Calculate D matrix
    D = differences / (distances + 1e-6)

    # Calculate G matrix
    G = ship_ground_gaussians(x, grounds, alpha, b=b)

    # Calculate Collection matrix
    C = ship_ground_gaussians(x, grounds, sigma, b=b)

    # Calculate A matrix
    A = attraction_matrix(U, G, D, F)

    # Calculate H
    H = harbor_attraction(U, x, h)


    S = np.sum(A, axis=1).reshape(-1, 2)

    S = S + H

    S_normal = S / (np.linalg.norm(S, axis=1, keepdims=True) + 1e-6)

    F_prime = calculate_F_prime(t, F, C, rate)

    x_prime = calculate_x_prime(U, S_normal)
    # print(np.linalg.norm(x_prime, axis=1))

          
    Dxs = Dxs_matrix(x, grounds, x, G, U, F, distances, differences, alpha, h)

    p_prime = calculate_p_prime(p, pf, S, U, Dxs, ship_diffences, differences, distances, rate, sigma, beta)

    pf_prime = calculate_p_f_prime(t, pf, C, rate)

    # print(p_prime)

    # Flatten x_prime and F_prime and concatenate them into a single 1D array
    # return np.concatenate((x_prime.flatten(), F_prime.flatten(), np.ones_like(p_prime).flatten()))
    return np.concatenate((x_prime.flatten(), F_prime.flatten(), p_prime.flatten(), pf_prime.flatten()))
def state(t, y, params):
    alpha, sigma, beta, rate, U, grounds, h, x_size, F_size, x_shape= params
    b = 1
    x = y[:x_size].reshape(x_shape)  # reshape x back to its original shape
    F = y[x_size:x_size+F_size]

    differences = grounds - x[:, np.newaxis]
    distances = np.linalg.norm(differences, axis=2, keepdims=True)

    # ship_distances = np.linalg.norm(ship_diffences, axis=2)

    # Calculate D matrix
    D = differences / (distances + 1e-6)

    # Calculate G matrix
    G = ship_ground_gaussians(x, grounds, alpha, b=b)

    # Calculate Collection matrix
    C = ship_ground_gaussians(x, grounds, sigma, b=b)

    # Calculate A matrix
    A = attraction_matrix(U, G, D, F)

    # Calculate H
    H = harbor_attraction(U, x, h)


    S = np.sum(A, axis=1).reshape(-1, 2)

    S = S + H

    S_normal = S / (np.linalg.norm(S, axis=1, keepdims=True) + 1e-6)

    F_prime = calculate_F_prime(t, F, C, rate)

    x_prime = calculate_x_prime(U, S_normal)

    return np.concatenate((x_prime.flatten(), F_prime.flatten()))
def costate(t, y, params):
    all_x, all_F, i, alpha, sigma, beta, rate, U, grounds, h, p_size, pf_size, p_shape = params
    b = 1
    p = y[:p_size].reshape(p_shape)  # reshape x back to its original shape
    pf = y[p_size:p_size+pf_size]

    x = all_x[i]
    F = all_F[i]

    differences = grounds - x[:, np.newaxis]
    distances = np.linalg.norm(differences, axis=2, keepdims=True)

    ship_diffences = x[:, np.newaxis] - x

    # ship_distances = np.linalg.norm(ship_diffences, axis=2)

    # Calculate D matrix
    D = differences / (distances + 1e-6)

    # Calculate G matrix
    G = ship_ground_gaussians(x, grounds, alpha, b=b)

    # Calculate Collection matrix
    C = ship_ground_gaussians(x, grounds, sigma, b=b)

    # Calculate A matrix
    A = attraction_matrix(U, G, D, F)

    # Calculate H
    H = harbor_attraction(U, x, h)


    S = np.sum(A, axis=1).reshape(-1, 2)

    # S = S + H


    Dxs = Dxs_matrix(x, grounds, x, G, U, F, distances, differences, alpha, h)

    p_prime = calculate_p_prime(p, pf, S, U, Dxs, ship_diffences, differences, distances, rate, sigma, beta)

    pf_prime = calculate_p_f_prime(t, pf, C, rate)

    return np.concatenate((p_prime.flatten(), pf_prime.flatten()))
alpha = .001
sigma = .01
beta = 1
rate = .01

x_max, y_max = 200, 200
n_ships = 20
n_grounds = 30
t_final = 500

# h is the harbor position
h = np.array([x_max / 2, 0])

grounds = generate_fishing_grounds(n_grounds, x_max, y_max)
ships = generate_ships(n_ships, x_max, y_max)
# print(ships.shape)
# ships = generate_ships_at_harbor(h, n_ships)

# put the grounds in the middle of both hemispheres

# put the ships at the harbor

# initialize U as all ones
U = np.ones((ships.shape[0], grounds.shape[0] + 2))

# initialize U with normal distribution, mean 1, std 0.1
U = np.random.normal(1, .25, (ships.shape[0], grounds.shape[0] + 2))
U = np.ones((ships.shape[0], grounds.shape[0] + 2))

# set the last column of U to 0
U[:,-2] = 0
U[:,-1] = .5

# initialize F as all ones
F = np.ones(grounds.shape[0])

# x is ship positions
x = ships



# Get the sizes of x and F before flattening
x_size = np.prod(x.shape)
x_shape = x.shape
F_size = np.prod(F.shape)

# Get the original shape of x

# Flatten x and F into a single 1D array
y0 = np.concatenate((x.flatten(), F))



state_params = (alpha, sigma, beta, rate, U, grounds, h, x_size, F_size, x_shape)

y_sol = solve_ivp(state, [0, t_final], y0, args=(state_params,), t_eval=np.linspace(0, t_final, t_final))

# Extract ship positions from the solution
all_x = y_sol.y[:x_size].reshape(-1, *x_shape)

# Extract F values from the solution
all_F = y_sol.y[x_size:x_size+F_size].T



# initialize p
p_init = np.full((n_ships, 2), 100)

# initialize pf, same size as F
pf_init = np.ones_like(F)
p_shape = p_init.shape
p_size = np.prod(p_init.shape)

# Flatten p and pf into a single 1D array
p0 = np.concatenate((p_init.flatten(), pf_init))

costate_params = (all_x, all_F, 0, alpha, sigma, beta, rate, U, grounds, h, p_size, F_size, p_shape)

p_sol = solve_ivp(costate, [0, t_final], p0, args=(costate_params,), t_eval=np.linspace(0, t_final, t_final))


